## Prepare data

In [ ]:
from StomataPy.data_preparation.stomata_data_preparation import Data4Stomata
input_dir = 'Data//Lab//Training//Stomata//Peels//Labels'
#Data4Stomata(input_dir=input_dir, aim='object detection').data4training()
Data4Stomata(input_dir=input_dir, 
             aim='object detection',
             new_width=2560,
             new_height=1920,
             sahi_slices=3,
             sahi_overlap_ratio=0.7).data4training()

In [ ]:
import os
input_dir = 'Data//Data//Publications//2017 Jayakody et al//Original'



## sort pore

In [1]:
from stomata_py.core.isat import *
input_dir = 'Data//Training//Stomata//Peels//Kidney//Labels'
# input_dir = 'Data//Training//Stomata//Peels//Kidney//A.thaliana Sabrina 2-pulse 150 white light missing time points'
sort_pore(input_dir)

In [4]:
import os  # interact with the operating system
import cv2  # OpenCV
import torch  # PyTorch
from skimage.measure import label, regionprops  # for using instance segmentation results as detected objects
from mmdet.utils import register_all_modules as mmdet_utils_register_all_modules  # register mmdet modules
from mmdet.apis import init_detector as mmdet_apis_init_detector  # initialize mmdet model
from mmdet.apis import inference_detector as mmdet_apis_inference_detector  # mmdet inference detector

class MMDET2ISAT:
    """Convert mmdet prediction to ISAT json files"""
    def __init__(self,
                 input_dir: str,
                 output_dir: str,
                 instance_detector_config_path: str,
                 instance_detector_weight_path: str,
                 instance_detector_threshold: float = 0.6,
                 ):
        self.input_dir = os.path.normpath(input_dir)  # input directory
        self.output_dir = os.path.normpath(output_dir)   # output folder name
        self.instance_detector_config_path = instance_detector_config_path  # instance segmentation config path
        self.instance_detector_weight_path = instance_detector_weight_path  # instance segmentation weight path
        self.instance_detector_threshold = instance_detector_threshold  # instance segmentation threshold

        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # use GPU if available
        image_types = ['.jpg', '.jpeg', '.png', '.tif', '.tiff', '.bmp', '.gif', '.ico', '.jfif', '.webp']  # supported image types
        file_names = [name for name in os.listdir(input_dir) if any(name.lower().endswith(file_type) for file_type in image_types)]  # image files only
        file_paths = [os.path.join(input_dir, name) for name in file_names]  # get image files paths
        self.images = [cv2.cvtColor(cv2.imread(file_path), cv2.COLOR_BGR2RGB) for file_path in file_paths]
        
        mmdet_utils_register_all_modules(init_default_scope=False)  # initialize mmdet scope
        self.instance_detector = mmdet_apis_init_detector(self.instance_detector_config_path, self.instance_detector_weight_path, device=device)  # initialize a instance detector from config file
    
    def toisat(self):
        for image in self.images:
            instance_predictions = mmdet_apis_inference_detector(self.instance_detector, image)  # inference image(s) with the instance detector.
            instance_indices = [torch.where(instance_predictions.pred_instances.scores > self.instance_detector_threshold)[0]]  # get indices of detected instances
            instance_masks = [instance_predictions.pred_instances.masks[instance_indices].cpu().numpy()]  # gte the masks of detected objects
            print(len(instance_masks))

MMDET2ISAT(input_dir='ISAT_SAM//example//images',
           output_dir='ISAT_SAM//example//images',
           instance_detector_config_path='mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco.py',
           instance_detector_weight_path='mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco_20220504_001756-c9d0c4f2.pth').toisat()

Loads checkpoint by local backend from path: mask2former_swin-s-p4-w7-224_8xb2-lsj-50e_coco_20220504_001756-c9d0c4f2.pth
1
1
1
1
1
1
